In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from optuna.integration import TFKerasPruningCallback
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint

In [ ]:
def load_data(data_path):
    data=pd.read_csv(data_path)
    X = data.iloc[: ,:-1].to_numpy()
    Y = data['labels'].to_numpy()

    X = X.reshape(X.shape[0], X.shape[1], 1)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

    return x_train, x_test, y_train, y_test

In [ ]:
# Assuming load_data is a function defined elsewhere
x_train, x_test, y_train, y_test = load_data("../data/data_proccesed_50.csv")

checkpoint = ModelCheckpoint(
    filepath=r"../model/best_optimalized.h5",  
    monitor='val_accuracy',        
    verbose=0,                 
    save_best_only=True,       
    mode='max'                 
)

def create_model(trial):
    lstm_units = trial.suggest_int('lstm_units', 32, 128)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    l1_reg = trial.suggest_float('l1_reg', 1e-5, 1e-2, log=True)
    l2_reg = trial.suggest_float('l2_reg', 1e-5, 1e-2, log=True)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    use_batch_norm = trial.suggest_categorical('use_batch_norm', [True, False])
    add_dense_before = trial.suggest_categorical('add_dense_before', [True, False])
    add_dense_after = trial.suggest_categorical('add_dense_after', [True, False])
    
    if add_dense_before:
        dense_units_before = trial.suggest_int('dense_units_before', 16, 64)
    
    if add_dense_after:
        dense_units_after = trial.suggest_int('dense_units_after', 16, 64)

    normalizer = layers.Normalization()
    normalizer.adapt(x_train)
    
    model = models.Sequential()
    model.add(layers.Input(shape=(x_train.shape[1], x_train.shape[2])))
    model.add(normalizer)
    
    if add_dense_before:
        model.add(layers.Dense(dense_units_before, activation='relu'))

    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.LSTM(lstm_units, activation="tanh", 
                          kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg)))
    model.add(layers.Dropout(dropout_rate))
    
    if add_dense_after:
        model.add(layers.Dense(dense_units_after, activation='relu'))

    model.add(layers.Dense(1, activation="sigmoid"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )
    
    return model

def objective(trial):
    model = create_model(trial)

    history = model.fit(
        x_train, y_train,
        epochs=10,
        batch_size=64,
        validation_data=(x_test, y_test),
        callbacks=[checkpoint,TFKerasPruningCallback(trial, 'val_accuracy')],
        verbose=0 
    )

    val_loss, val_accuracy = model.evaluate(x_test, y_test, verbose=0)
    return val_accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



best_trial = study.best_trial
model = create_model(best_trial)
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(x_test, y_test)
)

model.save('best_model.h5')

# Plot model architecture
plot_file = 'model.png'
plot_model(model, to_file=plot_file, show_shapes=True, show_layer_names=True)

# Display the model plot
plt.figure(figsize=(10, 15))
img = plt.imread(plot_file)
plt.imshow(img)
plt.axis('off')
plt.show()